In [ ]:
const handleAction = async (userInput) => {
  setIsProcessing(true);
  setError(null);
  setStatusMessage(null);
  const processingSteps = [];
  
  try {
    const action = userConfig.buttons[0];
    console.log(`Handling action: ${action.label}, User input: ${userInput}`);
    const content = await getEmailContent();
    const payload = {
      userId: userConfig.userId,
      emailContent: content,
      prompt: userInput,
    };

    console.log(`Sending request to: ${action.apiEndpoint}`);
    const response = await fetch(action.apiEndpoint, {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(payload),
    });

    if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
    
    const reader = response.body.getReader();
    const decoder = new TextDecoder();

    while (true) {
      const { value, done } = await reader.read();
      if (done) break;
      const decodedChunk = decoder.decode(value, { stream: true });
      const steps = decodedChunk.split('\n\n')
        .filter(step => step.trim() !== '')
        .map(step => step.replace('data: ', ''));
      
      for (const step of steps) {
        if (step.startsWith('Processing')) {
          processingSteps.push(step);
          setChatMessages(prevMessages => [
            ...prevMessages,
            { role: 'system', content: step }
          ]);
        } else {
          const wrappedContent = wrapInHtml(step);
          setChatMessages(prevMessages => [
            ...prevMessages,
            { role: 'assistant', content: wrappedContent }
          ]);
        }
      }
    }
  } catch (e) {
    console.error(`Error in handleAction: ${e.message}`);
    setError(`Failed to process your request. Please try again.`);
  } finally {
    setIsProcessing(false);
  }
};

In [ ]:
<div className={styles.chatMessages}>
  {chatMessages.map((message, index) => (
    <div
      key={index}
      className={`${styles.chatMessage} ${
        message.role === "user" 
          ? styles.userMessage 
          : message.role === "system" 
            ? styles.systemMessage 
            : styles.assistantMessage
      }`}
    >
      {message.role === "user" || message.role === "system" ? (
        message.content
      ) : (
        <div dangerouslySetInnerHTML={{ __html: message.content }} />
      )}
    </div>
  ))}
</div>

In [ ]:
systemMessage: {
  alignSelf: 'center',
  backgroundColor: tokens.colorNeutralBackground3,
  color: tokens.colorNeutralForeground3,
  fontStyle: 'italic',
},